In [1]:
import numpy as np
import math
import sklearn.preprocessing


In [2]:
from numpy import genfromtxt
my_data = genfromtxt('train.csv', delimiter=',')  
my_data = my_data[1:]
labels = my_data[:,0]
images = my_data[:,1:]
print(labels.shape)
print(images.shape)

(42000,)
(42000, 784)


In [3]:
images /= 255

In [4]:
test_images = images[0:4200]
test_labels = labels[0:4200]

In [5]:
train_images = images[4200:]
train_labels = labels[4200:]

In [6]:
test_labels_one_hot = np.zeros([4200,10])
train_labels_one_hot = np.zeros([4200,10])

In [7]:
label_binarizer = sklearn.preprocessing.LabelBinarizer()
label_binarizer.fit(range(max(test_labels.astype(np.int))+1))
test_labels_one_hot = label_binarizer.transform(test_labels).astype(np.float32)
train_labels_one_hot = label_binarizer.transform(train_labels).astype(np.float32)

In [8]:
print(test_images.shape)
print(test_labels_one_hot.shape)
print(train_images.shape)
print(train_labels_one_hot.shape)

(4200, 784)
(4200, 10)
(37800, 784)
(37800, 10)


In [9]:
print(train_labels_one_hot[0])
print(train_labels_one_hot[1])
print(train_labels_one_hot[2])

[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
[ 0.  0.  0.  0.  0.  0.  0.  1.  0.  0.]
[ 0.  1.  0.  0.  0.  0.  0.  0.  0.  0.]


In [10]:
b

NameError: name 'b' is not defined

In [21]:
def compute_layer(W, b, x):
    return np.matmul(W,x) + b

In [22]:
def sigmoid(z):
    return 1/(1 + np.exp(-1 * z))

In [23]:
def loss(a, y):
    return -(np.multiply(y,np.log(a)) + np.multiply((1 - y),np.log(1 - a)))

In [24]:
def cost(results):
    m = results.shape[0]
    return 1/m * results.sum(axis=0)

$J(W,b) = \frac{1}{m} \sum loss_i$

In [161]:

results = np.ndarray([10,10])
for i in range(10):
    z = compute_layer(W, b, train_images[i])
    y_hat = sigmoid(z)
    results[i] = y_hat

In [162]:
losses = np.ndarray([10,10])
for i in range(10):
    losses[i] = loss(results[i], train_labels_one_hot[i])

In [163]:
costs = cost(losses)

In [164]:
costs

array([ 0.68293351,  0.71871   ,  0.71057105,  0.66928756,  0.68944609,
        0.67641162,  0.69804161,  0.68635451,  0.71139254,  0.69239997])

In [152]:
gradientW = np.zeros(W.shape)
learning_rate = 0.1
num_batches = 100
batch_size = 10

In [155]:
batches = np.ndarray([num_batches,batch_size,784])
label_batches = np.ndarray([num_batches,batch_size,10])

In [25]:
import random

In [125]:
def make_batches(num_batches, batch_size, train_images, train_labels):
    batch_images = np.ndarray([num_batches, batch_size, 784])
    batch_labels = np.ndarray([num_batches, batch_size, 10])
    for i in range(num_batches):
        for j in range(batch_size):
            k = random.randint(0,30000)
            batch_images[i,j] = train_images[k]
            batch_labels[i,j] = train_labels[k]
    return (batch_images, batch_labels)

In [136]:
def train_1_layer_batchs(W, b, batch_images, batch_labels, learning_rate):
    num_batches = batch_images.shape[0]
    batch_size = batch_images.shape[1]
    W_updated = W
    b_updated = b
    for i in range(num_batches):
        imgs = batch_images[i]
        lbls = batch_labels[i]
        costs = np.ndarray([10])
        dw = np.ndarray([10,784])
        for j in range(batch_size):
            x = imgs[j]
            y = lbls[j]
            z = compute_layer(W_updated, b_updated, x)
            a = sigmoid(z)
            costs += a - y
            dw += np.tile(x, (10,1))
        # Compute Gradient
        m = batch_size
        costs = costs / m
        dw = dw
        dw = np.multiply(np.tile(costs, (784,1)).transpose(), dw)
        db = costs
        # Backprop
        W_updated = W_updated - learning_rate * dw
        b_updated = b_updated - learning_rate * db
        return (W_updated,b_updated)

In [137]:
def test_model(W,b,test_images, test_labels):
    num_correct = 0
    num_tested = 0
    for i in range(test_images.shape[0]):
        img = test_images[i]
        lbl = test_labels[i]
        z = compute_layer(W, b, img)
        a = sigmoid(z)
        num_tested += 1
        if(np.argmax(a) == np.argmax(lbl)):
            num_correct += 1
    accuracy = num_correct / num_tested
    print("Accuracy: " + str(accuracy))

In [138]:
W = np.random.uniform(-0.01,0.01,size=[10,784])
b = np.zeros([10])
test_model(W,b,test_images,test_labels_one_hot)

Accuracy: 0.10428571428571429


In [139]:
(imgs, lbls) = make_batches(10000, 10, train_images, train_labels_one_hot)

In [140]:
(W,b) = train_1_layer_batchs(W, b, imgs, lbls, 0.005)

In [141]:
test_model(W,b,test_images,test_labels_one_hot)

Accuracy: 0.10119047619047619


$w:w - a(dJ/dw)$
$\partial J/\partial dw = dJ/d$

In [128]:
def train_1_layer(W, b, images, labels, learning_rate):
    W_updated = W
    b_updated = b
    num_samples = images.shape[0]
    for i in range(num_samples):
        x = images[i]
        y = labels[i]
        z = compute_layer(W_updated, b_updated, x)
        a = sigmoid(z)
        l = loss(a, y)
        dw_l = np.tile(x, (10,1))
        dz = a - y
        dz_w = np.tile(dz, (784,1)).transpose()
        dw = np.multiply(dz_w, dw_l)
        W_updated = W_updated - learning_rate * dw
        db = dz
        b_updated = b_updated - learning_rate * db
    return (W_updated, b_updated)

In [116]:
W = np.random.uniform(-0.01,0.01,size=[10,784])
b = np.zeros([10])

In [117]:
test_model(W,b,test_images,test_labels_one_hot)

Accuracy: 0.06285714285714286


In [122]:
(W,b) = train_1_layer(W, b, train_images, train_labels_one_hot, 0.005)

In [123]:
 test_model(W,b,test_images,test_labels_one_hot)

Accuracy: 0.9064285714285715


In [146]:
def train_2_layer(W1, b1, W2, b2 images, labels, learning_rate):
    W_updated = W
    b_updated = b
    num_samples = images.shape[0]
    for i in range(num_samples):
        x = images[i]
        y = labels[i]
        z1 = compute_layer(W1, b1, x)
        a1 = sigmoid(z1)
        z2 = compute_layer(W2, b2, a1)
        a2 = sigmoid(z2)
        l = loss(a2, y)
        dw1_l = np.tile(x, (10,1))
        dz = a - y
        dz_w = np.tile(dz, (784,1)).transpose()
        dw = np.multiply(dz_w, dw_l)
        W_updated = W_updated - learning_rate * dw
        db = dz
        b_updated = b_updated - learning_rate * db
    return (W_updated, b_updated)

In [153]:
def test_model_2_layer(W1, b1, W2, b2, test_images, test_labels):
    num_correct = 0
    num_tested = 0
    for i in range(test_images.shape[0]):
        img = test_images[i]
        lbl = test_labels[i]
        z1 = compute_layer(W1, b1, img)
        a1 = sigmoid(z1)
        z2 = compute_layer(W2, b2, a1)
        a = sigmoid(z2)
        num_tested += 1
        if(np.argmax(a) == np.argmax(lbl)):
            num_correct += 1
    accuracy = num_correct / num_tested
    print("Accuracy: " + str(accuracy))

In [154]:
W1 = np.random.uniform(-0.01,0.01,size=[100,784])
b1 = np.zeros([100])
W2 = np.random.uniform(-0.01,0.01,size=[10,100])
b2 = np.zeros([10])
test_model_2_layer(W1,b1,W2,b2,test_images,test_labels_one_hot)

Accuracy: 0.10809523809523809
